#Rag Thresholding

In [ ]:
import os 
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import xgboost as xgb
import imutils 
from imutils import paths
from skimage import data, segmentation, color, filters
from skimage.transform import resize
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.model_selection import train_test_split

base_path = 'C:/Users/Farhan/Desktop/Pattern Project/Plant_sample100'
categories = ['Cherry___healthy', 'Cherry_Powdery_mildew', 'Corn___healthy', 
              'Corn_Common_rust', 'Grape___healthy', 'Grape_Isariopsis_Leaf_Spot']
f_names = []

### Reading File Name 

In [ ]:
for category in categories:
    fold_plant = os.path.join(base_path, category)
    file_names = os.listdir(fold_plant)
    full_path = [os.path.join(fold_plant, file_name) for file_name in file_names]
    f_names.append(full_path)

### Reading Images

In [ ]:
images = []
for names in f_names:
    one_category_images = [cv2.imread(name) for name in names if (cv2.imread(name)) is not None]
    images.append(one_category_images)
print('Number of images for each category:', [len(f) for f in images])

### Identifying Minimum Shape for Images

In [ ]:
for i,imgs in enumerate(images):
    shapes = [img.shape for img in imgs]
    widths = [shape[0] for shape in shapes]
    heights = [shape[1] for shape in shapes]
    print('%d,%d is the min shape for %s' % (np.min(widths), np.min(heights), categories[i]))
print('\n')
def bgr2rgb(img):
    return cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)

### Resize Image

In [ ]:
def image_to_feature_vector(images, size=(128, 128)):
    return cv2.resize(images, size).flatten()

### 3D Histogram

In [ ]:
def extract_color_histogram(imgs, bins=(16, 16, 16)):
    hsv = cv2.cvtColor(imgs, cv2.COLOR_BGR2RGB)
    h_gram = cv2.calcHist([hsv], [0, 1, 2], None, bins,[0, 180, 0, 256, 0, 256])
    if imutils.is_cv2():
        h_gram = cv2.normalize(h_gram)
    else:
        cv2.normalize(h_gram, h_gram)
    return h_gram.flatten()

 ##   Segmentation with Rag Thresholding

In [ ]:
labels = []
rawImages = []
features = []

imagePaths = list(paths.list_images(base_path))

for (i, imagePath) in enumerate(imagePaths):
    
    fig, axes = plt.subplots(ncols=3, figsize=(15, 3.5))
    ax = axes.ravel()
    ax[0] = plt.subplot(1, 3, 1)
    ax[1] = plt.subplot(1, 3, 2)
    ax[2] = plt.subplot(1, 3, 3) 
    
    image = cv2.imread(imagePath)

    ax[0].imshow(bgr2rgb(image))
    ax[0].set_title('Original')
    ax[0].axis('off')
    
    label = imagePath.split(os.path.sep)[-1].split(".")[0]

    #   segmentation with Rag_thresholding..................................................................
    labels1 = segmentation.slic(image, compactness=30, n_segments=400)
    out = color.label2rgb(labels1, image, kind='avg')

    ax[1].imshow(out)
    ax[1].set_title('Segmented picture')
    ax[1].axis('off')

    h_gram = extract_color_histogram(out)
    pixels = image_to_feature_vector(out)
    rawImages.append(pixels)                   # features
    labels.append(label)
    features.append(h_gram)

    ax[2].hist(out.ravel(),256,[0,256], color='r')
    ax[2].set_title('Histogram')
        
    plt.show()

rawImages = np.array(rawImages)               # show some information on the memory consumed by the raw images matrix and features matrix
labels = np.array(labels)
features = np.array(features)

print("[INFO] pixels matrix:   {:.2f} MB".format(rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f} MB\n".format(features.nbytes / (1024 * 1000.0)))

###Train-Test Split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(rawImages, labels, test_size=0.25, random_state=22)
(train_Feat, test_Feat, train_Labels, test_Labels) = train_test_split(features, labels, test_size=0.25, random_state=22)

##Decision Tree Classifier:

In [ ]:
Dclf = DecisionTreeClassifier(criterion="entropy", max_depth=3)
Dclf1 = DecisionTreeClassifier(criterion="entropy", max_depth=3)

Dclf.fit(X_train,y_train)
Dclf1.fit(train_Feat,train_Labels)

Dy_pred = Dclf.predict(X_test)
Dy_pred1=Dclf1.predict(test_Feat)
print("\nDecision Tree Classifier: \n")

result2 = confusion_matrix(y_test, Dy_pred)
print("Confusion Matrix:\n")
print(result2)

result3 = classification_report(y_test, Dy_pred)
print("Classification Report:")
print(result3)

D_acc=metrics.accuracy_score(y_test, Dy_pred)
print("Accuracy of decision tree: {:.2f} %".format(D_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, Dy_pred1)*100))


##Random Forest Classifier:

In [ ]:
clf=RandomForestClassifier(n_estimators=90)
clf1=RandomForestClassifier(n_estimators=90)

clf.fit(X_train,y_train)
clf1.fit(train_Feat,train_Labels)

r_pred = clf.predict(X_test)
r_pred1 = clf1.predict(test_Feat)

print("\nRandom Forest Classifier: \n")
result = confusion_matrix(y_test, r_pred)
print("Confusion Matrix: \n")
print(result)

result1 = classification_report(y_test, r_pred)
print("Classification Report: \n")
print(result1)
R_acc=(metrics.accuracy_score(y_test, r_pred))

print("Accuracy of Random Forest:{:.2f} %".format(R_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, r_pred1)*100))


##Naive Bayes Classifier:

In [ ]:
gnb = GaussianNB()
gnb1 = GaussianNB()

gnb.fit(X_train,y_train)
gnb1.fit(train_Feat,train_Labels)

Ny_pred = gnb.predict(X_test)
Ny_pred1=gnb1.predict(test_Feat)
print("\nNaive Bayes Classifier: \n")

result2 = confusion_matrix(y_test, Ny_pred)
print("Confusion Matrix:\n")
print(result2)

result3 = classification_report(y_test, Ny_pred)
print("Classification Report:")
print(result3)

N_acc=metrics.accuracy_score(y_test, Ny_pred)
print("Accuracy of Naive Bayes Classifier: {:.2f} %".format(N_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, Ny_pred1)*100))

##SVM Classifier:

In [ ]:
Sclf = svm.SVC(kernel='linear')
Sclf1 = svm.SVC(kernel='linear')

Sclf.fit(X_train,y_train)
Sclf1.fit(train_Feat,train_Labels)

Sy_pred = Sclf.predict(X_test)
Sy_pred1=Sclf1.predict(test_Feat)
print("\nSVM Classifier: \n")
result2 = confusion_matrix(y_test, Sy_pred)
print("Confusion Matrix:\n")
print(result2)

result3 = classification_report(y_test, Sy_pred)
print("Classification Report:")
print(result3)

S_acc=metrics.accuracy_score(y_test, Sy_pred)
print("Accuracy of SVM Classifier: {:.2f} %".format(S_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, Sy_pred1)*100))


##xgBoost classifier:

In [ ]:
xg_cl = xgb.XGBClassifier(n_estimators=10,objective ='binary:logistic',colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10,)
xg_cl1 = xgb.XGBClassifier(n_estimators=10,objective ='binary:logistic',colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10,)

xg_cl.fit(X_train,y_train)
xg_cl1.fit(train_Feat,train_Labels)

xgy_pred = xg_cl.predict(X_test)
xgy_pred1=xg_cl1.predict(test_Feat)
print("\nxgBoost Classifier: \n")
result2 = confusion_matrix(y_test, xgy_pred)

print("Confusion Matrix:\n")
print(result2)

result3 = classification_report(y_test, xgy_pred)
print("Classification Report:")
print(result3)

xg_acc=metrics.accuracy_score(y_test, xgy_pred)
print("Accuracy of xgBoost Classifier: {:.2f} %".format(xg_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, xgy_pred1)*100))

##KNN Classifier:

In [ ]:
neighbors = [1, 3, 5, 7, 9]
k_pred = []

for k in neighbors:
    print(f"\nKNN classifier ( k = {k})\n")
    knn = KNeighborsClassifier(n_neighbors= k)
    knn.fit(X_train,y_train)
    
    y_predi = knn.predict(X_test)
    prediction = metrics.accuracy_score(y_test, y_predi)
    k_pred.append(prediction)
        
    CM= confusion_matrix(y_test,y_predi)
    print("Confusion Matrix:\n")
    print(CM)
    classi=classification_report(y_test,y_predi)
    print("Classification Report:")
    print(classi)

    knn.fit(train_Feat,train_Labels)
    k_acc=knn.score(test_Feat,test_Labels)

    print("Accuracy of KNN: {:.1f} %".format(prediction * 100))
    print(f"Accuracy of features : {k_acc * 100} %")

##  Plotting Accuracy

In [ ]:
height = [D_acc*100, R_acc*100, N_acc*100, S_acc*100, xg_acc*100, 
          k_pred[0]*100, k_pred[1]*100, k_pred[2]*100, k_pred[3]*100, k_pred[4]*100]
bars = ('Decision tree', 'Random', 'Naive Bayes','SVM', 'xGBoost', 
        'KNN_1', 'KNN_3', 'KNN_5', 'KNN_7', 'KNN_9')
y_pos = np.arange(len(bars))

plt.figure(figsize=(20, 12))
plt.bar(y_pos, height, color=['red', 'palegreen', 'green', 'blue', 'yellow', 'brown', 'magenta', 'pink', 'magenta', 'pink'])
plt.xticks(y_pos, bars)
plt.title('Accuracy Comparision', size = 30)
plt.show()

#Region Rag

In [ ]:
import os 
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import xgboost as xgb
import imutils 
from imutils import paths
from skimage import data, segmentation, color, filters
from skimage.transform import resize
from skimage.future import graph
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.model_selection import train_test_split


base_path = 'C:/Users/Farhan/Desktop/Pattern Project/Plant_sample100'
categories = ['Cherry___healthy', 'Cherry_Powdery_mildew', 'Corn___healthy', 
              'Corn_Common_rust', 'Grape___healthy', 'Grape_Isariopsis_Leaf_Spot']
f_names = []

# reading file name 
for category in categories:
    fold_plant = os.path.join(base_path, category)
    file_names = os.listdir(fold_plant)
    full_path = [os.path.join(fold_plant, file_name) for file_name in file_names]
    f_names.append(full_path)

# reading images
images = []
for names in f_names:
    one_category_images = [cv2.imread(name) for name in names if (cv2.imread(name)) is not None]
    images.append(one_category_images)
print('Number of images for each category:', [len(f) for f in images])

# identifying minimum shape for images
for i,imgs in enumerate(images):
    shapes = [img.shape for img in imgs]
    widths = [shape[0] for shape in shapes]
    heights = [shape[1] for shape in shapes]
    print('%d,%d is the min shape for %s' % (np.min(widths), np.min(heights), categories[i]))
print('\n')
def bgr2rgb(img):
    return cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)
# resize image
def image_to_feature_vector(images, size=(128, 128)):
    return cv2.resize(images, size).flatten()
# 3D histogram
def extract_color_histogram(imgs, bins=(16, 16, 16)):
    hsv = cv2.cvtColor(imgs, cv2.COLOR_BGR2RGB)
    h_gram = cv2.calcHist([hsv], [0, 1, 2], None, bins,[0, 180, 0, 256, 0, 256])
    if imutils.is_cv2():
        h_gram = cv2.normalize(h_gram)
    else:
        cv2.normalize(h_gram, h_gram)
    return h_gram.flatten()

def weight_boundary(graph, src, dst, n):
    default = {'weight': 0.0, 'count': 0}

    count_src = graph[src].get(n, default)['count']
    count_dst = graph[dst].get(n, default)['count']

    weight_src = graph[src].get(n, default)['weight']
    weight_dst = graph[dst].get(n, default)['weight']

    count = count_src + count_dst
    return {
        'count': count,
        'weight': (count_src * weight_src + count_dst * weight_dst)/count
    }

def merge_boundary(graph, src, dst):
    pass

labels = []
rawImages = []
features = []

imagePaths = list(paths.list_images(base_path))

for (i, imagePath) in enumerate(imagePaths):
    
    fig, axes = plt.subplots(ncols=3, figsize=(15, 3.5))
    ax = axes.ravel()
    ax[0] = plt.subplot(1, 3, 1)
    ax[1] = plt.subplot(1, 3, 2)
    ax[2] = plt.subplot(1, 3, 3) 
    
    image = cv2.imread(imagePath)

    ax[0].imshow(bgr2rgb(image))
    ax[0].set_title('Original')
    ax[0].axis('off')
    
    label = imagePath.split(os.path.sep)[-1].split(".")[0]
    
    edges = filters.sobel(color.rgb2gray(image))
    labelss = segmentation.slic(image, compactness=30, n_segments=400)
    g = graph.rag_boundary(labelss, edges)

    graph.show_rag(labelss, g, image)
    plt.title('Initial RAG')

    labels2 = graph.merge_hierarchical(labelss, g, thresh=0.08, rag_copy=False,in_place_merge=True, merge_func=merge_boundary,weight_func=weight_boundary)

    graph.show_rag(labelss, g, image)
    plt.title('RAG after hierarchical merging')
    
    out = color.label2rgb(labels2, image, kind='avg')
    ax[1].imshow(out)
    ax[1].set_title('Segmented pic')
    ax[1].axis('off')

    h_gram = extract_color_histogram(out)
    pixels = image_to_feature_vector(out)
    rawImages.append(pixels)                   # features
    labels.append(label)
    features.append(h_gram)

    ax[2].hist(out.ravel(),256,[0,256], color='r')
    ax[2].set_title('Histogram')
        
    plt.show()
    

rawImages = np.array(rawImages)               # show some information on the memory consumed by the raw images matrix and features matrix
labels = np.array(labels)
features = np.array(features)

print("[INFO] pixels matrix:   {:.2f} MB".format(rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f} MB\n".format(features.nbytes / (1024 * 1000.0)))

X_train, X_test, y_train, y_test = train_test_split(rawImages, labels, test_size=0.25, random_state=22)
(train_Feat, test_Feat, train_Labels, test_Labels) = train_test_split(features, labels, test_size=0.25, random_state=22)


#Decision Tree Classifier:............................................................................

Dclf = DecisionTreeClassifier(criterion="entropy", max_depth=3)
Dclf1 = DecisionTreeClassifier(criterion="entropy", max_depth=3)

Dclf.fit(X_train,y_train)
Dclf1.fit(train_Feat,train_Labels)

Dy_pred = Dclf.predict(X_test)
Dy_pred1=Dclf1.predict(test_Feat)
print("\nDecision Tree Classifier: \n")

result2 = confusion_matrix(y_test, Dy_pred)
print("Confusion Matrix:\n")
print(result2)

result3 = classification_report(y_test, Dy_pred)
print("Classification Report:")
print(result3)

D_acc=metrics.accuracy_score(y_test, Dy_pred)
print("Accuracy of decision tree: {:.2f} %".format(D_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, Dy_pred1)*100))

#Random Forest Classifier:..................................................................................

clf=RandomForestClassifier(n_estimators=90)
clf1=RandomForestClassifier(n_estimators=90)

clf.fit(X_train,y_train)
clf1.fit(train_Feat,train_Labels)

r_pred = clf.predict(X_test)
r_pred1 = clf1.predict(test_Feat)

print("\nRandom Forest Classifier: \n")
result = confusion_matrix(y_test, r_pred)
print("Confusion Matrix: \n")
print(result)

result1 = classification_report(y_test, r_pred)
print("Classification Report: \n")
print(result1)
R_acc=(metrics.accuracy_score(y_test, r_pred))

print("Accuracy of Random Forest:{:.2f} %".format(R_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, r_pred1)*100))



#Naive Bayes Classifier:........................................................................

gnb = GaussianNB()
gnb1 = GaussianNB()

gnb.fit(X_train,y_train)
gnb1.fit(train_Feat,train_Labels)

Ny_pred = gnb.predict(X_test)
Ny_pred1=gnb1.predict(test_Feat)
print("\nNaive Bayes Classifier: \n")

result2 = confusion_matrix(y_test, Ny_pred)
print("Confusion Matrix:\n")
print(result2)

result3 = classification_report(y_test, Ny_pred)
print("Classification Report:")
print(result3)

N_acc=metrics.accuracy_score(y_test, Ny_pred)
print("Accuracy of Naive Bayes Classifier: {:.2f} %".format(N_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, Ny_pred1)*100))

#SVM Classifier:.....................................................................................

Sclf = svm.SVC(kernel='linear')
Sclf1 = svm.SVC(kernel='linear')

Sclf.fit(X_train,y_train)
Sclf1.fit(train_Feat,train_Labels)

Sy_pred = Sclf.predict(X_test)
Sy_pred1=Sclf1.predict(test_Feat)
print("\nSVM Classifier: \n")
result2 = confusion_matrix(y_test, Sy_pred)
print("Confusion Matrix:\n")
print(result2)

result3 = classification_report(y_test, Sy_pred)
print("Classification Report:")
print(result3)

S_acc=metrics.accuracy_score(y_test, Sy_pred)
print("Accuracy of SVM Classifier: {:.2f} %".format(S_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, Sy_pred1)*100))


#xgBoost classifier:.........................................................................


xg_cl = xgb.XGBClassifier(n_estimators=10,objective ='binary:logistic',colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10,)
xg_cl1 = xgb.XGBClassifier(n_estimators=10,objective ='binary:logistic',colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10,)

xg_cl.fit(X_train,y_train)
xg_cl1.fit(train_Feat,train_Labels)

xgy_pred = xg_cl.predict(X_test)
xgy_pred1=xg_cl1.predict(test_Feat)
print("\nxgBoost Classifier: \n")
result2 = confusion_matrix(y_test, xgy_pred)

print("Confusion Matrix:\n")
print(result2)

result3 = classification_report(y_test, xgy_pred)
print("Classification Report:")
print(result3)

xg_acc=metrics.accuracy_score(y_test, xgy_pred)
print("Accuracy of xgBoost Classifier: {:.2f} %".format(xg_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, xgy_pred1)*100))

#Knn Classifier:..................................................................................

neighbors = [1, 3, 5, 7, 9]
k_pred = []

for k in neighbors:
    print(f"\nKNN classifier ( k = {k})\n")
    knn = KNeighborsClassifier(n_neighbors= k)
    knn.fit(X_train,y_train)
    
    y_predi = knn.predict(X_test)
    prediction = metrics.accuracy_score(y_test, y_predi)
    k_pred.append(prediction)
        
    CM= confusion_matrix(y_test,y_predi)
    print("Confusion Matrix:\n")
    print(CM)
    classi=classification_report(y_test,y_predi)
    print("Classification Report:")
    print(classi)

    knn.fit(train_Feat,train_Labels)
    k_acc=knn.score(test_Feat,test_Labels)

    print("Accuracy of KNN: {:.1f} %".format(prediction * 100))
    print(f"Accuracy of features : {k_acc * 100} %")
        
#  Plotting Accuracy
height = [D_acc*100, R_acc*100, N_acc*100, S_acc*100, xg_acc*100, 
          k_pred[0]*100, k_pred[1]*100, k_pred[2]*100, k_pred[3]*100, k_pred[4]*100]
bars = ('Decision tree', 'Random', 'Naive Bayes','SVM', 'xGBoost', 
        'KNN_1', 'KNN_3', 'KNN_5', 'KNN_7', 'KNN_9')
y_pos = np.arange(len(bars))

plt.figure(figsize=(20, 12))
plt.bar(y_pos, height, color=['red', 'palegreen', 'green', 'blue', 'yellow', 'brown', 'magenta', 'pink', 'magenta', 'pink'])
plt.xticks(y_pos, bars)
plt.title('Accuracy Comparision', size = 30)
plt.show()


#Grab Cut

In [ ]:
import os 
import cv2
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import xgboost as xgb
import imutils 
from imutils import paths
from skimage import data, segmentation, color, filters
from skimage.transform import resize
from sklearn import metrics
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn import svm
from sklearn.model_selection import train_test_split


base_path = 'C:/Users/Farhan/Desktop/Pattern Project/Plant_sample100'
categories = ['Cherry___healthy', 'Cherry_Powdery_mildew', 'Corn___healthy', 
              'Corn_Common_rust', 'Grape___healthy', 'Grape_Isariopsis_Leaf_Spot']
f_names = []

# reading file name 
for category in categories:
    fold_plant = os.path.join(base_path, category)
    file_names = os.listdir(fold_plant)
    full_path = [os.path.join(fold_plant, file_name) for file_name in file_names]
    f_names.append(full_path)

# reading images
images = []
for names in f_names:
    one_category_images = [cv2.imread(name) for name in names if (cv2.imread(name)) is not None]
    images.append(one_category_images)
print('Number of images for each category:', [len(f) for f in images])

# identifying minimum shape for images
for i,imgs in enumerate(images):
    shapes = [img.shape for img in imgs]
    widths = [shape[0] for shape in shapes]
    heights = [shape[1] for shape in shapes]
    print('%d,%d is the min shape for %s' % (np.min(widths), np.min(heights), categories[i]))
print('\n')
def bgr2rgb(img):
    return cv2.cvtColor(img.copy(), cv2.COLOR_BGR2RGB)
# resize image
def image_to_feature_vector(images, size=(128, 128)):
    return cv2.resize(images, size).flatten()
# 3D histogram
def extract_color_histogram(imgs, bins=(16, 16, 16)):
    hsv = cv2.cvtColor(imgs, cv2.COLOR_BGR2RGB)
    h_gram = cv2.calcHist([hsv], [0, 1, 2], None, bins,[0, 180, 0, 256, 0, 256])
    if imutils.is_cv2():
        h_gram = cv2.normalize(h_gram)
    else:
        cv2.normalize(h_gram, h_gram)
    return h_gram.flatten()

labels = []
rawImages = []
features = []

imagePaths = list(paths.list_images(base_path))


for (i, imagePath) in enumerate(imagePaths):
    
    fig, axes = plt.subplots(ncols=3, figsize=(15, 3.5))
    ax = axes.ravel()
    ax[0] = plt.subplot(1, 3, 1)
    ax[1] = plt.subplot(1, 3, 2)
    ax[2] = plt.subplot(1, 3, 3) 
    
    image = cv2.imread(imagePath)

    ax[0].imshow(bgr2rgb(image))
    ax[0].set_title('Original')
    ax[0].axis('off')
    
    label = imagePath.split(os.path.sep)[-1].split(".")[0]

#   segmentation with Grab_cut..................................................................

    mask = np.zeros(image.shape[:2],np.uint8)
    bgdModel = np.zeros((1,65),np.float64)
    fgdModel = np.zeros((1,65),np.float64)

    rect = (3,3,950,950)
    cv2.grabCut(image,mask,rect,bgdModel,fgdModel,3,cv2.GC_INIT_WITH_RECT)

    mask2 = np.where((mask==2)|(mask==0),0,1).astype('uint8')
    g_cut = image*mask2[:,:,np.newaxis]

    ax[1].imshow(g_cut)
    ax[1].set_title('Segmented pic')
    ax[1].axis('off')

    h_gram = extract_color_histogram(g_cut)
    pixels = image_to_feature_vector(g_cut)
    rawImages.append(pixels)                   # features
    labels.append(label)
    features.append(h_gram)

    ax[2].hist(g_cut.ravel(),256,[0,256], color='r')
    ax[2].set_title('Histogram')
        
    plt.show()


rawImages = np.array(rawImages)               # show some information on the memory consumed by the raw images matrix and features matrix
labels = np.array(labels)
features = np.array(features)

print("[INFO] pixels matrix:   {:.2f} MB".format(rawImages.nbytes / (1024 * 1000.0)))
print("[INFO] features matrix: {:.2f} MB\n".format(features.nbytes / (1024 * 1000.0)))

X_train, X_test, y_train, y_test = train_test_split(rawImages, labels, test_size=0.25, random_state=22)
(train_Feat, test_Feat, train_Labels, test_Labels) = train_test_split(features, labels, test_size=0.25, random_state=22)


#Decision Tree Classifier:............................................................................

Dclf = DecisionTreeClassifier(criterion="entropy", max_depth=3)
Dclf1 = DecisionTreeClassifier(criterion="entropy", max_depth=3)

Dclf.fit(X_train,y_train)
Dclf1.fit(train_Feat,train_Labels)

Dy_pred = Dclf.predict(X_test)
Dy_pred1=Dclf1.predict(test_Feat)
print("\nDecision Tree Classifier: \n")

result2 = confusion_matrix(y_test, Dy_pred)
print("Confusion Matrix:\n")
print(result2)

result3 = classification_report(y_test, Dy_pred)
print("Classification Report:")
print(result3)

D_acc=metrics.accuracy_score(y_test, Dy_pred)
print("Accuracy of decision tree: {:.2f} %".format(D_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, Dy_pred1)*100))

#Random Forest Classifier:..................................................................................

clf=RandomForestClassifier(n_estimators=90)
clf1=RandomForestClassifier(n_estimators=90)

clf.fit(X_train,y_train)
clf1.fit(train_Feat,train_Labels)

r_pred = clf.predict(X_test)
r_pred1 = clf1.predict(test_Feat)

print("\nRandom Forest Classifier: \n")
result = confusion_matrix(y_test, r_pred)
print("Confusion Matrix: \n")
print(result)

result1 = classification_report(y_test, r_pred)
print("Classification Report: \n")
print(result1)
R_acc=(metrics.accuracy_score(y_test, r_pred))

print("Accuracy of Random Forest:{:.2f} %".format(R_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, r_pred1)*100))



#Naive Bayes Classifier:........................................................................

gnb = GaussianNB()
gnb1 = GaussianNB()

gnb.fit(X_train,y_train)
gnb1.fit(train_Feat,train_Labels)

Ny_pred = gnb.predict(X_test)
Ny_pred1=gnb1.predict(test_Feat)
print("\nNaive Bayes Classifier: \n")

result2 = confusion_matrix(y_test, Ny_pred)
print("Confusion Matrix:\n")
print(result2)

result3 = classification_report(y_test, Ny_pred)
print("Classification Report:")
print(result3)

N_acc=metrics.accuracy_score(y_test, Ny_pred)
print("Accuracy of Naive Bayes Classifier: {:.2f} %".format(N_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, Ny_pred1)*100))

#SVM Classifier:.....................................................................................

Sclf = svm.SVC(kernel='linear')
Sclf1 = svm.SVC(kernel='linear')

Sclf.fit(X_train,y_train)
Sclf1.fit(train_Feat,train_Labels)

Sy_pred = Sclf.predict(X_test)
Sy_pred1=Sclf1.predict(test_Feat)
print("\nSVM Classifier: \n")
result2 = confusion_matrix(y_test, Sy_pred)
print("Confusion Matrix:\n")
print(result2)

result3 = classification_report(y_test, Sy_pred)
print("Classification Report:")
print(result3)

S_acc=metrics.accuracy_score(y_test, Sy_pred)
print("Accuracy of SVM Classifier: {:.2f} %".format(S_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, Sy_pred1)*100))


#xgBoost classifier:.........................................................................


xg_cl = xgb.XGBClassifier(n_estimators=10,objective ='binary:logistic',colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10,)
xg_cl1 = xgb.XGBClassifier(n_estimators=10,objective ='binary:logistic',colsample_bytree = 0.3, learning_rate = 0.1,
                max_depth = 5, alpha = 10,)

xg_cl.fit(X_train,y_train)
xg_cl1.fit(train_Feat,train_Labels)

xgy_pred = xg_cl.predict(X_test)
xgy_pred1=xg_cl1.predict(test_Feat)
print("\nxgBoost Classifier: \n")
result2 = confusion_matrix(y_test, xgy_pred)

print("Confusion Matrix:\n")
print(result2)

result3 = classification_report(y_test, xgy_pred)
print("Classification Report:")
print(result3)

xg_acc=metrics.accuracy_score(y_test, xgy_pred)
print("Accuracy of xgBoost Classifier: {:.2f} %".format(xg_acc * 100))
print("Accuracy of features : {:.2f} %".format(metrics.accuracy_score(test_Labels, xgy_pred1)*100))

#Knn Classifier:..................................................................................

neighbors = [1, 3, 5, 7, 9]
k_pred = []

for k in neighbors:
    print(f"\nKNN classifier ( k = {k})\n")
    knn = KNeighborsClassifier(n_neighbors= k)
    knn.fit(X_train,y_train)
    
    y_predi = knn.predict(X_test)
    prediction = metrics.accuracy_score(y_test, y_predi)
    k_pred.append(prediction)
        
    CM= confusion_matrix(y_test,y_predi)
    print("Confusion Matrix:\n")
    print(CM)
    classi=classification_report(y_test,y_predi)
    print("Classification Report:")
    print(classi)

    knn.fit(train_Feat,train_Labels)
    k_acc=knn.score(test_Feat,test_Labels)

    print("Accuracy of KNN: {:.1f} %".format(prediction * 100))
    print(f"Accuracy of features : {k_acc * 100} %")
        
#  Plotting Accuracy
height = [D_acc*100, R_acc*100, N_acc*100, S_acc*100, xg_acc*100, 
          k_pred[0]*100, k_pred[1]*100, k_pred[2]*100, k_pred[3]*100, k_pred[4]*100]
bars = ('Decision tree', 'Random', 'Naive Bayes','SVM', 'xGBoost', 
        'KNN_1', 'KNN_3', 'KNN_5', 'KNN_7', 'KNN_9')
y_pos = np.arange(len(bars))

plt.figure(figsize=(20, 12))
plt.bar(y_pos, height, color=['red', 'palegreen', 'green', 'blue', 'yellow', 'brown', 'magenta', 'pink', 'magenta', 'pink'])
plt.xticks(y_pos, bars)
plt.title('Accuracy Comparision', size = 30)
plt.show()